# <center><font color='green'>**Capstone Project**</font></center>
## <center><font color='darkred'>Canon - EMEA</center>
## <center>Matching Employees to Projects</center>
### <left>Saad Joiya</left>

<span style="font-size: 18px;">
    
**Project aim:**
We are trying to create an algorithm to automatically output a list of ideal employees based on project requirements. 

**Method**

*Employee and Project Feature Space:*

Define the features that would be used in both the employees and projects table. Next, we would create mock data with relevant data formats and realistic values for all features that mimics real world data closely.

*Score Calculation:*

We match relevant features from both tables based on different matching techniques depending on the feature data types and values to generate optimal scores for these feature subgroups. Next, we assign weights to these different feature subgroups to generate a final scoring. Finally, we use some features in post processing for ranking all relevant employees for each project. 
</span> 

# Importing Libraries

In [2]:
import pandas as pd
import random
import difflib
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime, timedelta

!pip install -q sentence-transformers
from sentence_transformers import SentenceTransformer, util


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Generating Projects and Roles

**For internal consistency within same instances, for some of the features we define these thematically controlled pools so feature values are consistent and as close as possible to real world scenarios. We used Canon's internal job postings and LinkedIN profile to manually gather these values.**

**Other fields which are global and would not be thematically controlled.**

**Six high-level business themes include:**

1. Technical
2. Sales
3. Marketing
4. HR
5. Legal
6. Consulting

**This ensures internal consistency between:**

1. Products involved
2. Skills
3. Certification
4. Integration Requirements
5. Project Summary and Employee Role
6. Scope and Deliverables

In [8]:
# Controlled Product Pools per Theme
product_pools = {
    "Technical": ["Workflow2000", "Print2.0", "AIScan", "CloudSuite", "IntegrationHub"],
    "Sales": ["CRM Pro", "Sales Enablement Suite", "Loyalty CRM", "SalesForce Light"],
    "Marketing": ["Digital Campaign Manager", "SEO Toolkit", "Content CMS", "Social Media Manager"],
    "HR": ["HRIS Plus", "Onboarding Suite", "Employee Experience Platform"],
    "Legal": ["Compliance Suite", "Contract Manager Pro", "Regulatory Tracker"],
    "Consulting": ["ERP Migration Tool", "Business Analysis Framework", "Strategy Kit"]
}

# Controlled Skill Pools per Theme
skill_pools = {
    "Technical": ["Data Analysis", "Workflow Automation", "Cloud Services", "IT Infrastructure", "API Development"],
    "Sales": ["CRM Integration", "Negotiation", "Client Management", "Customer Relationship Management"],
    "Marketing": ["SEO Optimization", "Content Strategy", "Campaign Management", "Copywriting", "Branding"],
    "HR": ["Digital HR", "Organizational Development", "Talent Management", "Communication Skills"],
    "Legal": ["Contract Management", "Regulatory Knowledge", "Document Review", "Compliance Documentation"],
    "Consulting": ["Business Analysis", "Strategic Planning", "Workflow Optimization", "Project Management", "Change Management"]
}

# New: Controlled Certifications Pool per Theme
certification_pools = {
    "Technical": ["ITIL", "ISO 27001", "Microsoft Azure Certification"],
    "Sales": ["Certified Sales Professional (CSP)", "CRM Specialist Certification"],
    "Marketing": ["Digital Marketing Certification", "Google Ads Certification", "HubSpot Marketing Certification"],
    "HR": ["PMP", "SHRM-CP", "HR Analytics Certification"],
    "Legal": ["Certified Compliance Officer", "GDPR Certification", "Contract Law Certification"],
    "Consulting": ["PMP", "Six Sigma", "Agile Practitioner", "Business Analysis Certification"]
}

# New: Controlled Expertise Areas Pool per Theme
expertise_pools = {
    "Technical": ["Scripting", "API Integration", "Cloud Infrastructure", "Networking", "Cybersecurity"],
    "Sales": ["CRM Integration", "Sales Pipeline Automation", "Client Relationship Systems"],
    "Marketing": ["SEO Optimization", "Content Management Systems", "Social Media Integration"],
    "HR": ["HRIS Systems", "Employee Experience Platforms", "Organizational Development Systems"],
    "Legal": ["Document Archiving", "Contract Management Systems", "Regulatory Compliance Tools"],
    "Consulting": ["Strategic Planning", "Business Workflow Optimization", "ERP Systems Integration"]
}


# Predefined vocabularies
locations_master = ["Berlin", "Vienna", "London"]
work_flexibility_options = ["onsite", "remote", "hybrid"]
languages_master = ["English", "French", "German", "Italian"]
fluency_levels = ["A1", "A2", "B1", "B2", "C1", "C2"]
industries_master = ["Healthcare", "Education", "Finance", "Manufacturing", "Retail"]

**These employee roles and Project Summary/Scope are thematically controlled and based on manual internet research on Canon. We used Canon's job postings and LinkedIN to gather this information. For Demo purposes we would generate these using Google Gemini LLM. The Gemini would be provided with prompts using the same thematically controlled pools defined above to generate the descriptions**

In [11]:
# --- Define Employee Roles with Themes ---
employee_roles = [
    {"Role Name": "Solution Architect", "Role Description": "Designs high-level technical solutions for enterprise customers.", "Theme": "Technical"},
    {"Role Name": "Sales Account Manager", "Role Description": "Manages customer accounts and drives sales processes.", "Theme": "Sales"},
    {"Role Name": "Digital Marketing Specialist", "Role Description": "Executes online campaigns, SEO, and branding strategies.", "Theme": "Marketing"},
    {"Role Name": "Senior HR Manager", "Role Description": "Manages HR operations and employee relations.", "Theme": "HR"},
    {"Role Name": "Legal Counsel", "Role Description": "Provides legal support for contracts and compliance.", "Theme": "Legal"},
    {"Role Name": "IT Systems Engineer", "Role Description": "Maintains and optimizes internal IT infrastructure.", "Theme": "Technical"},
    {"Role Name": "Workflow Consultant", "Role Description": "Analyzes business processes and recommends workflow improvements.", "Theme": "Consulting"},
    {"Role Name": "Project Manager", "Role Description": "Oversees project delivery and coordinates cross-functional teams.", "Theme": "Consulting"},
    {"Role Name": "Data Analyst", "Role Description": "Analyzes data and delivers business insights.", "Theme": "Technical"},
    {"Role Name": "Customer Success Manager", "Role Description": "Supports post-sales success and client satisfaction.", "Theme": "Sales"},
    {"Role Name": "Field Support Engineer", "Role Description": "Provides onsite technical support for Canon products.", "Theme": "Technical"},
    {"Role Name": "Pre-Sales Engineer", "Role Description": "Prepares technical demos and solution proposals for prospects.", "Theme": "Sales"},
    {"Role Name": "Compliance Manager", "Role Description": "Ensures adherence to regulations and company standards.", "Theme": "Legal"},
    {"Role Name": "HR Business Partner", "Role Description": "Collaborates with leadership to align HR strategy.", "Theme": "HR"},
    {"Role Name": "Corporate Trainer", "Role Description": "Designs and delivers employee training programs.", "Theme": "HR"},
    {"Role Name": "Technical Support Specialist", "Role Description": "Resolves technical issues reported by customers.", "Theme": "Technical"},
    {"Role Name": "Content Creator", "Role Description": "Develops written, video, and visual content for marketing.", "Theme": "Marketing"},
    {"Role Name": "Solutions Support Consultant", "Role Description": "Provides technical guidance and second-line support for Canon solutions during and after customer deployment.", "Theme": "Technical"},
    {"Role Name": "Integration Developer", "Role Description": "Develops integrations between Canon products and third-party systems.", "Theme": "Technical"},
    {"Role Name": "Strategy Consultant", "Role Description": "Advises leadership on business growth and optimization strategies.", "Theme": "Consulting"},
]

# --- Define Project Summaries with Themes ---
project_summaries = [
    {"Project Summary": "Implement scalable workflow automation system", "Scope and Deliverables": "Deploy Workflow2000, integrate with client systems", "Theme": "Technical"},
    {"Project Summary": "CRM integration for loyalty program", "Scope and Deliverables": "Customize CRM modules and train sales team", "Theme": "Sales"},
    {"Project Summary": "Launch digital marketing portal", "Scope and Deliverables": "Create website, SEO, lead funnels", "Theme": "Marketing"},
    {"Project Summary": "HR digital onboarding system", "Scope and Deliverables": "Implement HRIS system, self-service portals", "Theme": "HR"},
    {"Project Summary": "Contract management system deployment", "Scope and Deliverables": "Deploy document archiving and e-signature workflows", "Theme": "Legal"},
    {"Project Summary": "Upgrade internal IT infrastructure", "Scope and Deliverables": "Replace old servers, migrate systems to cloud", "Theme": "Technical"},
    {"Project Summary": "Business workflow audit", "Scope and Deliverables": "Map processes and suggest automation improvements", "Theme": "Consulting"},
    {"Project Summary": "Manage ERP migration project", "Scope and Deliverables": "Deliver milestones for new ERP roll-out", "Theme": "Consulting"},
    {"Project Summary": "Data warehouse design", "Scope and Deliverables": "Create new analytics-ready database", "Theme": "Technical"},
    {"Project Summary": "Post-sale onboarding program", "Scope and Deliverables": "Develop client onboarding workflow", "Theme": "Sales"},
    {"Project Summary": "Onsite print solutions setup", "Scope and Deliverables": "Install Print2.0 platform for retail client", "Theme": "Technical"},
    {"Project Summary": "Pre-sales technical proof-of-concept setup", "Scope and Deliverables": "Build demo environments for prospects", "Theme": "Sales"},
    {"Project Summary": "Regulatory compliance documentation project", "Scope and Deliverables": "Standardize processes, deliver compliance documentation", "Theme": "Legal"},
    {"Project Summary": "Organizational culture development initiative", "Scope and Deliverables": "Conduct workshops, employee surveys", "Theme": "HR"},
    {"Project Summary": "Employee training", "Scope and Deliverables": "Give overview on Learning Management System (LMS)", "Theme": "HR"},
    {"Project Summary": "Customer remote support setup", "Scope and Deliverables": "Setup online ticketing and remote assistance systems", "Theme": "Technical"},
    {"Project Summary": "Content library migration", "Scope and Deliverables": "Migrate marketing content to new CMS", "Theme": "Marketing"},
    {"Project Summary": "Quality assurance framework rollout", "Scope and Deliverables": "Implement QA policies across departments", "Theme": "Technical"},
    {"Project Summary": "API and system integration project", "Scope and Deliverables": "Develop middleware for integration of ERP/CRM", "Theme": "Technical"},
    {"Project Summary": "Business strategy development program", "Scope and Deliverables": "Assist C-suite with market expansion strategy", "Theme": "Consulting"},
]

# --- Create DataFrames ---
roles_df = pd.DataFrame(employee_roles)
projects_df = pd.DataFrame(project_summaries)

In [13]:
roles_df.head()

,Role Name,Role Description,Theme
0,Solution Architect,Designs high-level technical solutions for ent...,Technical
1,Sales Account Manager,Manages customer accounts and drives sales pro...,Sales
2,Digital Marketing Specialist,"Executes online campaigns, SEO, and branding s...",Marketing
3,Senior HR Manager,Manages HR operations and employee relations.,HR
4,Legal Counsel,Provides legal support for contracts and compl...,Legal


**We introduce typos with in some of the projects fields to mimic real world human data entry errors**

**The functions below would generate Project and Employees table based on defined pools and criterias**

In [17]:
# --- Typo Function ---
def introduce_typo(text):
    if len(text) < 4:
        return text
    idx = random.randint(0, len(text) - 2)
    return text[:idx] + text[idx+1] + text[idx] + text[idx+2:]

# --- Smart Project Generator (Full Version) ---
def generate_smart_projects_full(n, project_templates):
    projects = []
    for i in range(n):
        proj = random.choice(project_templates)
        theme = proj["Theme"]

        products = [introduce_typo(p) if random.random() < 0.4 else p
                    for p in random.sample(product_pools[theme], k=min(len(product_pools[theme]), random.randint(1, 3)))]

        required_skills = {
            (introduce_typo(skill) if random.random() < 0.4 else skill): random.randint(5, 10)
            for skill in random.sample(skill_pools[theme], k=min(len(skill_pools[theme]), random.randint(2, 5)))
        }

        certifications = random.sample(certification_pools[theme], k=min(len(certification_pools[theme]), random.randint(1, 2)))
        expertise = random.sample(expertise_pools[theme], k=min(len(expertise_pools[theme]), random.randint(1, 2)))

        project_industry = introduce_typo(random.choice(industries_master)) if random.random() < 0.4 else random.choice(industries_master)

        project = {
            "ProjectID": f"P{i+1}",
            "Project Summary": proj["Project Summary"],
            "Scope and Deliverables": proj["Scope and Deliverables"],
            "Theme": theme,
            "Products Involved": products,
            "Required Skills and Expertise": required_skills,
            "Customer Preferences (Certifications)": certifications,
            "Integration Requirements (Expertise Areas)": expertise,
            "Customer Industry": project_industry,
            "Work Location": introduce_typo(random.choice(locations_master)) if random.random() < 0.3 else random.choice(locations_master),
            "Work Flexibility": random.choice(work_flexibility_options),
            "Languages Required": {introduce_typo(lang) if random.random() < 0.3 else lang: random.choice(fluency_levels)
                                   for lang in random.sample(languages_master, k=random.randint(1, 3))},
            "Complexity": random.randint(1, 10),
            "Effort": random.choice(list(range(20, 241, 10))),
            "Requested End": datetime.now().date() + timedelta(days=random.randint(15, 90))
        }

        projects.append(project)
    return pd.DataFrame(projects)

# --- Smart Employee Generator (Full Version) ---
def generate_smart_employees_full(n, role_templates):
    employees = []
    for i in range(n):
        emp = random.choice(role_templates)
        theme = emp["Theme"]

        product_experience = random.sample(product_pools[theme], k=min(len(product_pools[theme]), random.randint(1, 3)))
        core_competencies = {skill: random.randint(4, 10) for skill in random.sample(skill_pools[theme], k=min(len(skill_pools[theme]), random.randint(2, 5)))}
        certifications = random.sample(certification_pools[theme], k=min(len(certification_pools[theme]), random.randint(1, 2)))
        expertise = random.sample(expertise_pools[theme], k=min(len(expertise_pools[theme]), random.randint(1, 2)))

        industry_experience = random.sample(industries_master, k=random.randint(1, 3))

        employee = {
            "EmployeeID": f"E{i+1}",
            "Role Name": emp["Role Name"],
            "Role Description": emp["Role Description"],
            "Theme": theme,
            "Products Experience": product_experience,
            "Core Competencies": core_competencies,
            "External/Internal Certifications": certifications,
            "Expertise Areas": expertise,
            "Industry Experience": industry_experience,
            "Work Location": random.choice(locations_master),
            "Work Flexibility": random.choice(work_flexibility_options),
            "Languages Known": {lang: random.choice(fluency_levels) 
                                for lang in random.sample(languages_master, k=random.randint(1, 3))},
            "Available From": datetime.now().date() + timedelta(days=random.randint(0, 30)),
            "Weekly Availability in Hours": random.choice([10, 20, 30, 40]),

             #Bonus Soft Skill Fields:
            "Cultural Awareness": random.randint(1, 10),
            "Problem Solving": random.randint(1, 10),
            "Leadership": random.randint(1, 10),
        }

        employees.append(employee)
    return pd.DataFrame(employees)


In [19]:
smart_projects_df = generate_smart_projects_full(20, project_summaries)
smart_employees_df = generate_smart_employees_full(20, employee_roles)

In [21]:
smart_projects_df.head()

,ProjectID,Project Summary,Scope and Deliverables,Theme,Products Involved,Required Skills and Expertise,Customer Preferences (Certifications),Integration Requirements (Expertise Areas),Customer Industry,Work Location,Work Flexibility,Languages Required,Complexity,Effort,Requested End
0,P1,Upgrade internal IT infrastructure,"Replace old servers, migrate systems to cloud",Technical,"[IntegratinoHub, ClouduSite, Print2.0]","{'Workfolw Automation': 10, 'Cloud Services': ...","[ISO 27001, Microsoft Azure Certification]","[Cloud Infrastructure, Networking]",eRtail,Beriln,hybrid,{'Germna': 'C1'},9,110,2025-08-17
1,P2,Organizational culture development initiative,"Conduct workshops, employee surveys",HR,"[HRI SPlus, Employee Experience Platform]","{'DigitalH R': 7, 'Organizational Development'...","[HR Analytics Certification, PMP]",[Organizational Development Systems],Education,Lnodon,hybrid,"{'Italian': 'A1', 'French': 'B1'}",10,80,2025-07-23
2,P3,API and system integration project,Develop middleware for integration of ERP/CRM,Technical,"[IntegratioHnub, Workflow2000, Pirnt2.0]","{'Workflow Automation': 7, 'Cloud Services': 5}","[Microsoft Azure Certification, ITIL]",[Networking],Edcuation,Vienna,onsite,"{'French': 'C2', 'Germna': 'C2', 'Italian': 'C2'}",6,220,2025-07-06
3,P4,Post-sale onboarding program,Develop client onboarding workflow,Sales,[Sales Enablement Suite],"{'Client Management': 5, 'Negotiation': 10, 'C...","[Certified Sales Professional (CSP), CRM Speci...",[Sales Pipeline Automation],Retail,Vienna,onsite,"{'French': 'A1', 'Italian': 'C1'}",3,20,2025-08-13
4,P5,CRM integration for loyalty program,Customize CRM modules and train sales team,Sales,[Loyalty CRM],"{'CRM Inetgration': 10, 'Customer Relationship...","[CRM Specialist Certification, Certified Sales...",[Client Relationship Systems],Finance,London,remote,{'French': 'B1'},2,120,2025-08-17


In [23]:
smart_employees_df.head()

,EmployeeID,Role Name,Role Description,Theme,Products Experience,Core Competencies,External/Internal Certifications,Expertise Areas,Industry Experience,Work Location,Work Flexibility,Languages Known,Available From,Weekly Availability in Hours,Cultural Awareness,Problem Solving,Leadership
0,E1,Corporate Trainer,Designs and delivers employee training programs.,HR,[Onboarding Suite],"{'Talent Management': 6, 'Communication Skills...","[HR Analytics Certification, SHRM-CP]",[Employee Experience Platforms],[Manufacturing],Vienna,onsite,"{'French': 'C2', 'English': 'C2'}",2025-06-29,10,5,9,8
1,E2,Technical Support Specialist,Resolves technical issues reported by customers.,Technical,"[CloudSuite, Workflow2000, Print2.0]","{'Data Analysis': 7, 'Cloud Services': 9}","[ISO 27001, Microsoft Azure Certification]",[Scripting],"[Retail, Education]",Berlin,onsite,"{'French': 'C2', 'Italian': 'C2', 'English': '...",2025-06-27,30,10,10,1
2,E3,Strategy Consultant,Advises leadership on business growth and opti...,Consulting,"[Business Analysis Framework, ERP Migration To...","{'Workflow Optimization': 6, 'Strategic Planni...",[PMP],[Strategic Planning],"[Retail, Education, Finance]",London,remote,{'German': 'A1'},2025-06-16,40,9,5,10
3,E4,Solution Architect,Designs high-level technical solutions for ent...,Technical,"[CloudSuite, IntegrationHub, Workflow2000]","{'API Development': 4, 'Data Analysis': 6, 'Wo...","[ITIL, Microsoft Azure Certification]","[Cybersecurity, API Integration]",[Finance],Vienna,onsite,"{'French': 'C1', 'English': 'A1'}",2025-06-18,30,4,2,7
4,E5,Content Creator,"Develops written, video, and visual content fo...",Marketing,"[Digital Campaign Manager, Content CMS]","{'Campaign Management': 9, 'SEO Optimization':...","[HubSpot Marketing Certification, Google Ads C...",[SEO Optimization],"[Education, Manufacturing]",Vienna,onsite,"{'Italian': 'C2', 'French': 'B2'}",2025-06-21,30,5,7,9


# Creating Scoring Functions

**We define the fuzzy match criteria to be used in the matching of fields that have typos in them**

In [27]:
fuzzy_match_threshold = 0.7

In [29]:
# Normalizing text and doing fuzzy match
def normalize(text):
    return text.lower().strip()

def fuzzy_match(val1, val2, threshold = fuzzy_match_threshold):
    val1, val2 = normalize(val1), normalize(val2)
    return difflib.SequenceMatcher(None, val1, val2).ratio() >= threshold

### Product Matching

Here we will have a list of products in the projects table and similarly a list in the employees table. As the products involved field in projects data is human filled, we first use fuzzy matching to get rid of any mismatching with products experience column in employee table caused by spelling mistakes and then we match the columns. The coverage is calculated based on how many required products are possessed by the employee.  For example, if a project requires products AIScan, Print2.0 and Workflow2000 and employee knows only AIScan, the coverage would be 33%. If an employee knows any two of the above then 66% and in case of knowing all three or more, it would be 100%.

In [33]:
def product_score(project_products, employee_products):
    match_count = 0
    for p_prod in project_products:
        if any(fuzzy_match(p_prod, e_prod) for e_prod in employee_products):
            match_count += 1
    return match_count / len(project_products) if project_products else 0

### Location Matching with Work Flexibility Logic


This calculation assigns a location match score based on both:

1.	Work flexibility compatibility

2.	Location similarity (using fuzzy matching)

Note: As Locations are filled by Humans, to match them we would use fuzzy matching.


| Project Flexibility | Employee Flexibility | Location Match | Score | Explanation                                      |
|---------------------|----------------------|----------------|-------|--------------------------------------------------|
| remote              | Any                  | —              | 1.0   | Location irrelevant for remote work              |
| Any except remote   | Any                  | No             | 0.0   | Location mismatch or wrong flexibility           |
| onsite              | onsite               | Yes            | 1.0   | Perfect location and presence match              |
| onsite              | hybrid               | Yes            | 0.5   | Partial match; available some days onsite        |
| onsite              | remote               | Yes            | 0.0   | No physical presence at required location        |
| hybrid              | onsite               | Yes            | 1.0   | Can fully accommodate onsite days                |
| hybrid              | hybrid               | Yes            | 1.0   | Flex on both sides; good match                   |
| hybrid              | remote               | Yes            | 0.5   | Remote match possible but less ideal             |
ut less ideal


In [38]:
def location_score(project_location, project_flex, employee_location, employee_flex):
    if project_flex == "remote":
        return 1.0
    location_match = fuzzy_match(project_location, employee_location)
    
    if project_flex == "onsite":
        if employee_flex == "onsite" and location_match:
            return 1.0
        elif employee_flex == "hybrid" and location_match:
            return 0.5
        else:
            return 0.0
    elif project_flex == "hybrid":
        if employee_flex == "onsite" and location_match:
            return 1.0
        elif employee_flex == "hybrid" and location_match:
            return 1.0
        elif employee_flex == "remote" and location_match:
            return 0.5
        else:
            return 0.0
    return 0.0

### Language Matching and Fluency Scoring

We match project language requirements with employee language fluency, handling:

•	Typos/misspellings in language names (fuzzy matching)

•	Fluency level comparison using the CEFR scale

•	Finally, scoring based on coverage and fluency fit

Steps for calculating Language Score:

1. For each project language, find the best fuzzy match in an employee’s known languages
   
2. If a match is found:
 Compare CEFR levels using the cefr_scale

3. Score per language:
• If employee level ≥ required → score = 1.0
• Else → 1 - (diff / 6)

4. Calculate Average Fit:
The mean of all individual language scores per language (fluency comparison), only for matched languages.

5. Coverage = matched languages / total required

6. Final Score = coverage × average fit

In [43]:
cefr_scale = {"A1": 1, "A2": 2, "B1": 3, "B2": 4, "C1": 5, "C2": 6}

def best_fuzzy_match(input_lang, employee_langs, threshold= fuzzy_match_threshold):
    best_match = None
    best_score = 0
    for e_lang in employee_langs:
        score = difflib.SequenceMatcher(None, normalize(input_lang), normalize(e_lang)).ratio()
        if score > best_score:
            best_match = e_lang
            best_score = score
    return best_match if best_score >= threshold else None

def language_score(project_langs, employee_langs):
    matched = []
    for p_lang, p_level in project_langs.items():
        matched_lang = best_fuzzy_match(p_lang, employee_langs)
        if matched_lang:
            matched.append((p_lang, matched_lang, p_level, employee_langs[matched_lang]))

    if not matched:
        return 0.0

    coverage = len(matched) / len(project_langs)
    scores = []
    for _, _, p_level, e_level in matched:
        required = cefr_scale.get(p_level, 0)
        actual = cefr_scale.get(e_level, 0)
        if actual >= required:
            score = 1.0
        else:
            score = max(0, 1 - (required - actual) / 6)
        scores.append(score)
    avg_fit = sum(scores) / len(scores)
    return round(coverage * avg_fit, 2)




### Industry Matching Score

Here we will have a customer industry column in the projects table containing a category based on the industry of the client. Similarly, in the employee’s table we would have an Industry Experience column containing a list of different industries the employee is experienced in. As the industries in the projects data is human filled, we first use fuzzy matching to get rid of any mismatching with industry experience column in employee table caused by spelling mistakes and then we match the columns. The score would be 1 if the employee is experienced in the client industry and 0 otherwise. For example, if a project is from a “retail” clients and employee has experience with “Manufacturing”, “Education”, “Retail”, the score would be 1. If an employee knows “Manufacturing” and “Education” or any list excluding “Retail” the score would be 0.

In [47]:
def fuzzy_in_list(value, lst, threshold=fuzzy_match_threshold):
    for item in lst:
        if difflib.SequenceMatcher(None, value.lower(), item.lower()).ratio() >= threshold:
            return True
    return False

def industry_score(project_industry, employee_industries, threshold=fuzzy_match_threshold):
    return 1.0 if fuzzy_in_list(project_industry, employee_industries, threshold) else 0.0


### Required Skills Matching

Scoring is done on how well an employee's core competencies align with the skills required for a project, considering:

1.	Coverage — How many required skills they know.
2.	Expertise Fit — How experienced are they in those skills.
3.	Complexity Fit — How well their skills meet the challenge of the projec → capability / complexity


Steps for calculating Skill Score:

1. **Fuzzy Skill Matching**
Match each required skill (may contain typos) with the closest skill in the employee’s core competency using fuzzy logic.

2. **Coverage Score**
Calculate what fraction of required skills are present in the employee’s skill set.

Formula:
coverage = matched_skills / total_required_skills

3. **Expertise Fit**

For each matched skill, compare employee’s level to the required level:

-	If level is equal or higher → score = 1.0

-	If lower → score = 1 - (required - actual) / 10

-	Average these for expertise_fit.

4. **Capability Score**

Multiply coverage by expertise_fit

capability = coverage × expertise_fit

5. **Final Skill Score (Complexity Fit)**

Compare capability with project complexity rating (normalized as complexity / 10):

->	If capability ≥ complexity → score = 1.0

->	Else → capability / complexity


In [52]:
# Skill Match Score with fuzzy key match

def skill_match_score_with_fuzzy_keys(project_skills_dict, complexity, core_competency):
    matched_pairs = []
    for p_skill, required_level in project_skills_dict.items():
        match = best_fuzzy_match(p_skill, core_competency)
        if match:
            matched_pairs.append((p_skill, match, required_level, core_competency[match]))
    if not matched_pairs:
        return 0.0
    coverage = len(matched_pairs) / len(project_skills_dict)
    expertise_scores = []
    for _, _, required, actual in matched_pairs:
        score = 1.0 if actual >= required else 1 - (required - actual) / 10
        expertise_scores.append(score)
    expertise_fit = sum(expertise_scores) / len(expertise_scores)
    capability = coverage * expertise_fit
    complexity_target = complexity / 10
    return 1.0 if capability >= complexity_target else round(capability / complexity_target, 2)


### Customer Preferences (Certifications) Matching

Here we will have a list of certifications in the projects table that the client demands are  must have. Similarly, a list of certifications that the employee has completed would be in the employees table. As the certifications field in projects data is human filled, we first use fuzzy matching to get rid of any mismatching with certifications column in employee table caused by spelling mistakes and then we match the columns. The coverage is calculated based on how many required certifications are possessed by the employee. For example, if a project requires Microsoft Azure Certification and ISO 27001 and employee has only ISO 27001, the coverage would be 50%. If an employee knows both above (or additional) then 100%. If no certifications are required, the employee gets a full score (100%).  

In [56]:
# Certification Match Score (Set Overlap)
def certification_score(project_certs, employee_certs, threshold = fuzzy_match_threshold):
    if not project_certs:
        return 1.0  # if project does not require certifications, full score
    match_count = 0
    for p_cert in project_certs:
        if any(fuzzy_match(p_cert, e_cert) for e_cert in employee_certs):
            match_count += 1
    return match_count / len(project_certs)

### Expertise (Integration Requirement) Matching

Here we will have a list of expertise areas required for the project in the projects table Similarly, a list of employee expertise in the employee’s table. As this field in projects data is human filled, we first use fuzzy matching to get rid of any mismatching with expertise column in employee table caused by spelling mistakes and then we match the columns. The score is calculated based on the coverage of required expertise against possessed expertise, identical to the certification score calculation. 

In [60]:
# Expertise Areas Match Score (Set Overlap)
def expertise_score(project_expertise, employee_expertise, threshold = fuzzy_match_threshold):
    if not project_expertise:
        return 1.0  # if project does not require expertise, full score
    match_count = 0
    for p_area in project_expertise:
        if any(fuzzy_match(p_area, e_area) for e_area in employee_expertise):
            match_count += 1
    return match_count / len(project_expertise)

### Project Summary and Scope & Deliverables Matching with Employee Job Role

To evaluate how well an employee’s role matches the intent of a project, we compare the Project Summary and Scope and Deliverables fields with the employee’s Role Description. We use Hugging Face AI's transformer (BAAI/bge-large-en-v1.5) for vectorization and then use cosine similarity to compute the semantic closeness between the text pairs.We do this separately for Project Summary vs Role and Scope and Deliverables vs Role. 
The two similarity scores (one for the summary, one for the scope) are averaged to produce a final Job description match score, representing how relevant an employee’s responsibilities are to the project described.


| Comparison Pair                            | Description                                              | Output                     |
|--------------------------------------------|----------------------------------------------------------|----------------------------|
| Project Summary vs Role Description        | Measures how well the overall project aligns with the role | `summary_sim` (0–1)       |
| Scope and Deliverables vs Role Description | Measures how well the specific responsibilities align     | `scope_sim` (0–1)         |
| Final Job Description Score                | Average of the two scores                                 | (`summary_sim` + `scope_sim`) / 2 |


In [67]:

# Load the embedding model (only once)
embedding_model = SentenceTransformer("BAAI/bge-large-en-v1.5")

def text_similarity_bge(text1, text2):
    # Generate vector embeddings
    embeddings = embedding_model.encode([text1, text2], convert_to_tensor=True)

    # Compute cosine similarity
    similarity = util.cos_sim(embeddings[0], embeddings[1]).item()
    
    return round(similarity, 3)


### Availability Filtering

The Availability Score measures whether an employee can complete a project within the required time frame based on their availability start date and weekly working capacity.

Steps For scoring:

1.	Compare dates:

First we compare employee’s Available From date to project’s Requested End date. If the employee is available only after Requested End Date then the score is 0.0 (not available).

2.	Estimate working time:

-	Calculate total calendar days between employee’s Available From Date and Project’s Requested End Date.
-	We convert the calendar days to working days using a 5/7 multiplier (for a standard 5-day workweek).
-	Derive available working weeks by dividing working days by 5.
  
3.	Calculate required effort per week:

-	Required Weekly Effort = Project Effort / Available Weeks
  
4.	Score logic:

-	If Required Weekly Effort ≤ Employee Weekly Capacity, then we say the score is 1.0
  
-	Otherwise, the score is a decimal ratio (e.g. 0.7), based on how much of the requirement the employee can handle.
Additionally, an Available flag is created that is 1 if Availability Score is 1 and 0 otherwise. The reason to create “Availability Score” and “Available” flag separately are that if no instance has Available Flag as 1, we can consider high Availability Score employees for the project.


In [75]:
from datetime import datetime, timedelta

def availability_score(project_effort, project_end, employee_available_from, employee_weekly_capacity):
    """
    Calculate availability score based on required effort and actual employee availability.
    Uses working days logic:
    - Converts calendar days into estimated working days (5 per 7)
    - Converts those working days into work weeks
    - Checks if employee can handle effort in the available time
    Returns:
        1.0 if the employee is available and has enough capacity
        0.0 otherwise
    """
    if employee_available_from > project_end:
        return 0.0

    days_available = (project_end - employee_available_from).days
    if days_available <= 0:
        return 0.0

    working_days = days_available * (5 / 7)       # Approximate working days
    weeks_available = working_days / 5            # Convert to working weeks

    if weeks_available <= 0:
        return 0.0

    required_weekly_effort = project_effort / weeks_available

    if required_weekly_effort <= employee_weekly_capacity:
        return 1.0
    else:
        return round(employee_weekly_capacity / required_weekly_effort, 2)

# Creating Merged Table For Scoring All Employees by Project

In [78]:
smart_projects_df["key"] = 1
smart_employees_df["key"] = 1
merged_df = pd.merge(smart_projects_df, smart_employees_df, on="key").drop(columns="key")


In [80]:
merged_df.head()

,ProjectID,Project Summary,Scope and Deliverables,Theme_x,Products Involved,Required Skills and Expertise,Customer Preferences (Certifications),Integration Requirements (Expertise Areas),Customer Industry,Work Location_x,...,Expertise Areas,Industry Experience,Work Location_y,Work Flexibility_y,Languages Known,Available From,Weekly Availability in Hours,Cultural Awareness,Problem Solving,Leadership
0,P1,Upgrade internal IT infrastructure,"Replace old servers, migrate systems to cloud",Technical,"[IntegratinoHub, ClouduSite, Print2.0]","{'Workfolw Automation': 10, 'Cloud Services': ...","[ISO 27001, Microsoft Azure Certification]","[Cloud Infrastructure, Networking]",eRtail,Beriln,...,[Employee Experience Platforms],[Manufacturing],Vienna,onsite,"{'French': 'C2', 'English': 'C2'}",2025-06-29,10,5,9,8
1,P1,Upgrade internal IT infrastructure,"Replace old servers, migrate systems to cloud",Technical,"[IntegratinoHub, ClouduSite, Print2.0]","{'Workfolw Automation': 10, 'Cloud Services': ...","[ISO 27001, Microsoft Azure Certification]","[Cloud Infrastructure, Networking]",eRtail,Beriln,...,[Scripting],"[Retail, Education]",Berlin,onsite,"{'French': 'C2', 'Italian': 'C2', 'English': '...",2025-06-27,30,10,10,1
2,P1,Upgrade internal IT infrastructure,"Replace old servers, migrate systems to cloud",Technical,"[IntegratinoHub, ClouduSite, Print2.0]","{'Workfolw Automation': 10, 'Cloud Services': ...","[ISO 27001, Microsoft Azure Certification]","[Cloud Infrastructure, Networking]",eRtail,Beriln,...,[Strategic Planning],"[Retail, Education, Finance]",London,remote,{'German': 'A1'},2025-06-16,40,9,5,10
3,P1,Upgrade internal IT infrastructure,"Replace old servers, migrate systems to cloud",Technical,"[IntegratinoHub, ClouduSite, Print2.0]","{'Workfolw Automation': 10, 'Cloud Services': ...","[ISO 27001, Microsoft Azure Certification]","[Cloud Infrastructure, Networking]",eRtail,Beriln,...,"[Cybersecurity, API Integration]",[Finance],Vienna,onsite,"{'French': 'C1', 'English': 'A1'}",2025-06-18,30,4,2,7
4,P1,Upgrade internal IT infrastructure,"Replace old servers, migrate systems to cloud",Technical,"[IntegratinoHub, ClouduSite, Print2.0]","{'Workfolw Automation': 10, 'Cloud Services': ...","[ISO 27001, Microsoft Azure Certification]","[Cloud Infrastructure, Networking]",eRtail,Beriln,...,[SEO Optimization],"[Education, Manufacturing]",Vienna,onsite,"{'Italian': 'C2', 'French': 'B2'}",2025-06-21,30,5,7,9


# Scoring Each Employee per Project

**Based on all scoring logics discussed above, we create a final scoring table for each employee against each project**

**Note:**
*Here we also bring 3 bonus scores into the frame that were present in employees table namely Cultural Awareness Score, Leadership Score and Problem Solving Score. These would be factored directly into the final score**

In [84]:
scores = []

for _, row in merged_df.iterrows():
    # Text similarity from summary and scope to role description
    summary_sim = text_similarity_bge(row["Project Summary"], row["Role Description"])
    scope_sim = text_similarity_bge(row["Scope and Deliverables"], row["Role Description"])
    role_fit_score = round((summary_sim + scope_sim) / 2, 3)


    availability = availability_score(row["Effort"],row["Requested End"],row["Available From"],row["Weekly Availability in Hours"])
    
    scores.append({
        "ProjectID": row["ProjectID"],
        "EmployeeID": row["EmployeeID"],


        "Product Match Score": round(product_score(row["Products Involved"], row["Products Experience"]), 2),
        "Location Match Score": round(location_score(row["Work Location_x"], row["Work Flexibility_x"],
                                                     row["Work Location_y"], row["Work Flexibility_y"]), 2),
        "Language Match Score": language_score(row["Languages Required"], row["Languages Known"]),
        "Industry Match Score": industry_score(row["Customer Industry"], row["Industry Experience"]),
        "Skill Match Score": skill_match_score_with_fuzzy_keys(row["Required Skills and Expertise"], row["Complexity"], row["Core Competencies"]),


        "Certification Match Score": certification_score(row["Customer Preferences (Certifications)"], row["External/Internal Certifications"]),
        "Expertise Match Score": expertise_score(row["Integration Requirements (Expertise Areas)"], row["Expertise Areas"]),
        "Job Description Match Score": role_fit_score,
        "Cultural Awareness Score": row["Cultural Awareness"]/10,
        "Problem Solving Score": row["Problem Solving"]/10,
        "Leadership Score": row["Leadership"]/10,
        "Availability Score": availability,
        "Available": 1 if availability == 1.0 else 0,
        
    })

# Final dataframe
scored_df = pd.DataFrame(scores)
scored_df.head()


,ProjectID,EmployeeID,Product Match Score,Location Match Score,Language Match Score,Industry Match Score,Skill Match Score,Certification Match Score,Expertise Match Score,Job Description Match Score,Cultural Awareness Score,Problem Solving Score,Leadership Score,Availability Score,Available
0,P1,E1,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.458,0.5,0.9,0.8,0.64,0
1,P1,E2,0.67,1.0,0.00,1.0,0.40,1.0,0.0,0.536,1.0,1.0,0.1,1.00,1
2,P1,E3,0.00,0.0,0.33,1.0,0.13,0.0,0.0,0.452,0.9,0.5,1.0,1.00,1
3,P1,E4,0.67,0.0,0.00,0.0,0.73,0.5,0.0,0.522,0.4,0.2,0.7,1.00,1
4,P1,E5,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.387,0.5,0.7,0.9,1.00,1


In [85]:
merged_df = merged_df.merge(scored_df, how = 'left', left_on = ['ProjectID','EmployeeID'], right_on = ['ProjectID','EmployeeID'])

In [86]:
pd.set_option('display.max_columns', None)
pd.DataFrame(merged_df.head(100))

,ProjectID,Project Summary,Scope and Deliverables,Theme_x,Products Involved,Required Skills and Expertise,Customer Preferences (Certifications),Integration Requirements (Expertise Areas),Customer Industry,Work Location_x,Work Flexibility_x,Languages Required,Complexity,Effort,Requested End,EmployeeID,Role Name,Role Description,Theme_y,Products Experience,Core Competencies,External/Internal Certifications,Expertise Areas,Industry Experience,Work Location_y,Work Flexibility_y,Languages Known,Available From,Weekly Availability in Hours,Cultural Awareness,Problem Solving,Leadership,Product Match Score,Location Match Score,Language Match Score,Industry Match Score,Skill Match Score,Certification Match Score,Expertise Match Score,Job Description Match Score,Cultural Awareness Score,Problem Solving Score,Leadership Score,Availability Score,Available
0,P1,Upgrade internal IT infrastructure,"Replace old servers, migrate systems to cloud",Technical,"[IntegratinoHub, ClouduSite, Print2.0]","{'Workfolw Automation': 10, 'Cloud Services': ...","[ISO 27001, Microsoft Azure Certification]","[Cloud Infrastructure, Networking]",eRtail,Beriln,hybrid,{'Germna': 'C1'},9,110,2025-08-17,E1,Corporate Trainer,Designs and delivers employee training programs.,HR,[Onboarding Suite],"{'Talent Management': 6, 'Communication Skills...","[HR Analytics Certification, SHRM-CP]",[Employee Experience Platforms],[Manufacturing],Vienna,onsite,"{'French': 'C2', 'English': 'C2'}",2025-06-29,10,5,9,8,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.458,0.5,0.9,0.8,0.64,0
1,P1,Upgrade internal IT infrastructure,"Replace old servers, migrate systems to cloud",Technical,"[IntegratinoHub, ClouduSite, Print2.0]","{'Workfolw Automation': 10, 'Cloud Services': ...","[ISO 27001, Microsoft Azure Certification]","[Cloud Infrastructure, Networking]",eRtail,Beriln,hybrid,{'Germna': 'C1'},9,110,2025-08-17,E2,Technical Support Specialist,Resolves technical issues reported by customers.,Technical,"[CloudSuite, Workflow2000, Print2.0]","{'Data Analysis': 7, 'Cloud Services': 9}","[ISO 27001, Microsoft Azure Certification]",[Scripting],"[Retail, Education]",Berlin,onsite,"{'French': 'C2', 'Italian': 'C2', 'English': '...",2025-06-27,30,10,10,1,0.67,1.0,0.00,1.0,0.40,1.0,0.0,0.536,1.0,1.0,0.1,1.00,1
2,P1,Upgrade internal IT infrastructure,"Replace old servers, migrate systems to cloud",Technical,"[IntegratinoHub, ClouduSite, Print2.0]","{'Workfolw Automation': 10, 'Cloud Services': ...","[ISO 27001, Microsoft Azure Certification]","[Cloud Infrastructure, Networking]",eRtail,Beriln,hybrid,{'Germna': 'C1'},9,110,2025-08-17,E3,Strategy Consultant,Advises leadership on business growth and opti...,Consulting,"[Business Analysis Framework, ERP Migration To...","{'Workflow Optimization': 6, 'Strategic Planni...",[PMP],[Strategic Planning],"[Retail, Education, Finance]",London,remote,{'German': 'A1'},2025-06-16,40,9,5,10,0.00,0.0,0.33,1.0,0.13,0.0,0.0,0.452,0.9,0.5,1.0,1.00,1
3,P1,Upgrade internal IT infrastructure,"Replace old servers, migrate systems to cloud",Technical,"[IntegratinoHub, ClouduSite, Print2.0]","{'Workfolw Automation': 10, 'Cloud Services': ...","[ISO 27001, Microsoft Azure Certification]","[Cloud Infrastructure, Networking]",eRtail,Beriln,hybrid,{'Germna': 'C1'},9,110,2025-08-17,E4,Solution Architect,Designs high-level technical solutions for ent...,Technical,"[CloudSuite, IntegrationHub, Workflow2000]","{'API Development': 4, 'Data Analysis': 6, 'Wo...","[ITIL, Microsoft Azure Certification]","[Cybersecurity, API Integration]",[Finance],Vienna,onsite,"{'French': 'C1', 'English': 'A1'}",2025-06-18,30,4,2,7,0.67,0.0,0.00,0.0,0.73,0.5,0.0,0.522,0.4,0.2,0.7,1.00,1
4,P1,Upgrade internal IT infrastructure,"Replace old servers, migrate systems to cloud",Technical,"[IntegratinoHub, ClouduSite, Print2.0]","{'Workfolw Automation': 10, 'Cloud Services': ...","[ISO 27001, Microsoft Azure Certification]","[Cloud Infrastructure, Networking]",eRtail,Beriln,hybrid,{'Germna': 'C1'},9,110,2025-08-17,E5,Con

## Final Weighted Score

**In this section we calculate a final weighted score for each employee against each project based on the weightages assigned to each factor**

In [100]:
scored_df = merged_df.copy()

**Defining Weights for each score**

In [103]:
#weights sum up to 1 so we get score from 0 to 1
weights = {
    "Product Match Score": 0.1,
    "Location Match Score": 0.1,
    "Language Match Score": 0.1,
    "Industry Match Score": 0.075,
    "Skill Match Score": 0.2,
    "Certification Match Score": 0.1,
    "Expertise Match Score": 0.1,
    "Job Description Match Score": 0.15,
    "Cultural Awareness Score": 0.025,
    "Problem Solving Score": 0.025,
    "Leadership Score": 0.025
}


In [105]:
for col, weight in weights.items():
    scored_df[col + " Weighted"] = scored_df[col] * weight

scored_df["Final Score"] = scored_df[[col + " Weighted" for col in weights]].sum(axis=1)


**Filtering for employees based on availability score. This gives the user flexibility to choose preferred employees even with less than 1.0 availability scores**

In [108]:
filtered_df = scored_df.loc[scored_df['Availability Score'] >= 0.7]



**After filtering by availability score, we then sort all ProjectID then by final score to calculate Final Employee Rank per Project**

In [113]:
# Sort within each project
ranked = filtered_df.sort_values(["ProjectID", "Final Score"], ascending=[True, False])


In [117]:
ranked.head(100)

,ProjectID,Project Summary,Scope and Deliverables,Theme_x,Products Involved,Required Skills and Expertise,Customer Preferences (Certifications),Integration Requirements (Expertise Areas),Customer Industry,Work Location_x,Work Flexibility_x,Languages Required,Complexity,Effort,Requested End,EmployeeID,Role Name,Role Description,Theme_y,Products Experience,Core Competencies,External/Internal Certifications,Expertise Areas,Industry Experience,Work Location_y,Work Flexibility_y,Languages Known,Available From,Weekly Availability in Hours,Cultural Awareness,Problem Solving,Leadership,Product Match Score,Location Match Score,Language Match Score,Industry Match Score,Skill Match Score,Certification Match Score,Expertise Match Score,Job Description Match Score,Cultural Awareness Score,Problem Solving Score,Leadership Score,Availability Score,Available,Product Match Score Weighted,Location Match Score Weighted,Language Match Score Weighted,Industry Match Score Weighted,Skill Match Score Weighted,Certification Match Score Weighted,Expertise Match Score Weighted,Job Description Match Score Weighted,Cultural Awareness Score Weighted,Problem Solving Score Weighted,Leadership Score Weighted,Final Score
5,P1,Upgrade internal IT infrastructure,"Replace old servers, migrate systems to cloud",Technical,"[IntegratinoHub, ClouduSite, Print2.0]","{'Workfolw Automation': 10, 'Cloud Services': ...","[ISO 27001, Microsoft Azure Certification]","[Cloud Infrastructure, Networking]",eRtail,Beriln,hybrid,{'Germna': 'C1'},9,110,2025-08-17,E6,IT Systems Engineer,Maintains and optimizes internal IT infrastruc...,Technical,"[AIScan, Workflow2000]","{'Data Analysis': 10, 'Cloud Services': 5, 'AP...","[ISO 27001, Microsoft Azure Certification]","[Networking, API Integration]","[Finance, Manufacturing]",Berlin,hybrid,"{'French': 'B1', 'Italian': 'B2'}",2025-06-11,20,3,9,7,0.00,1.0,0.00,0.0,1.00,1.0,0.5,0.696,0.3,0.9,0.7,1.00,1,0.000,0.1,0.000,0.000,0.200,0.10,0.05,0.10440,0.0075,0.0225,0.0175,0.60190
7,P1,Upgrade internal IT infrastructure,"Replace old servers, migrate systems to cloud",Technical,"[IntegratinoHub, ClouduSite, Print2.0]","{'Workfolw Automation': 10, 'Cloud Services': ...","[ISO 27001, Microsoft Azure Certification]","[Cloud Infrastructure, Networking]",eRtail,Beriln,hybrid,{'Germna': 'C1'},9,110,2025-08-17,E8,Field Support Engineer,Provides onsite technical support for Canon pr...,Technical,[Workflow2000],"{'Workflow Automation': 10, 'Cloud Services': ...","[Microsoft Azure Certification, ISO 27001]",[Networking],"[Manufacturing, Education]",Berlin,onsite,"{'Italian': 'A2', 'English': 'A2', 'French': '...",2025-06-05,30,7,1,5,0.00,1.0,0.00,0.0,1.00,1.0,0.5,0.533,0.7,0.1,0.5,1.00,1,0.000,0.1,0.000,0.000,0.200,0.10,0.05,0.07995,0.0175,0.0025,0.0125,0.56245
1,P1,Upgrade internal IT infrastructure,"Replace old servers, migrate systems to cloud",Technical,"[IntegratinoHub, ClouduSite, Print2.0]","{'Workfolw Automation': 10, 'Cloud Services': ...","[ISO 27001, Microsoft Azure Certification]","[Cloud Infrastructure, Networking]",eRtail,Beriln,hybrid,{'Germna': 'C1'},9,110,2025-08-17,E2,Technical Support Specialist,Resolves technical issues reported by customers.,Technical,"[CloudSuite, Workflow2000, Print2.0]","{'Data Analysis': 7, 'Cloud Services': 9}","[ISO 27001, Microsoft Azure Certification]",[Scripting],"[Retail, Education]",Berlin,onsite,"{'French': 'C2', 'Italian': 'C2', 'English': '...",2025-06-27,30,10,10,1,0.67,1.0,0.00,1.0,0.40,1.0,0.0,0.536,1.0,1.0,0.1,1.00,1,0.067,0.1,0.000,0.075,0.080,0.10,0.00,0.08040,0.0250,0.0250,0.0025,0.55490
18,P1,Upgrade internal IT infrastructure,"Replace old servers, migrate systems to cloud",Technical,"[IntegratinoHub, ClouduSite, Print2.0]","{'Workfolw Automation': 10, 'Cloud Services': ...","[ISO 27001, Microsoft Azure Certification]","[Cloud Infrastructure, Networking]",eRtail,Beriln,hybrid,{'Germna': 'C1'},9,110,2025-08-17,E19,Technical Support Specialist,Resolves technical issues reported by customers.,Tech

In [119]:
ranked.to_csv('WeightedScores.csv')